<h1>Extracting base finetuned(with BD-SHS dataset) from the finetuned model(Did not run the code yet)<h1>

In [1]:
!pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00


In [3]:
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

In [4]:
#defining some hyperparameters
max_number_input_tokens=256
batch_size_training = 16
first_dropout_rate = 0.3
hidden_output = 768
bert_model_name = "sagorsarker/bangla-bert-base"
adam_opt_lr = 3e-5
scheduler_step = 1
scheduler_gamma = 0.8
epochs = 10
classes = 2
need_split_dataset=False

In [5]:
from google.colab import drive
drive.mount('/content/drive')
DirPath = ('/content/drive/My Drive/Test/')
Finetuned_model_path = DirPath+bert_model_name+"_CustomBertBengaliFullDataset6epoch885044valacc.pth"
CollectedDatasetFileName = "Final_data.csv"
CollectedDatasetPath = DirPath+"EMNLP/"+CollectedDatasetFileName
SplittedTrainFileName = "train.csv"
SplittedValFileName = "dev.csv"
SplittedTrainDataPath = DirPath+"EMNLP/"+SplittedTrainFileName
SplittedValDataPath = DirPath+"EMNLP/"+SplittedValFileName

Mounted at /content/drive


In [6]:
def interchange(df_train,pos,label):
  #setting the first sample to be with label '0'
  zero_index = df_train[df_train['label'] == label].index[0]
  first_index=pos
  # interchange the samples
  df_train.iloc[[first_index, zero_index]] = df_train.iloc[[zero_index, first_index]]
  return df_train

In [144]:
def balanceclasses(df_train):
  class_counts = df_train['label'].value_counts()
  min_count = class_counts.max()

  # Create new DataFrames for each class with fewer samples
  new_dfs = []
  for label, count in class_counts.items():
    if count == min_count:
        continue
    df_label = df_train[df_train['label'] == label]
    num_copies = min_count // count
    new_df_label = pd.concat([df_label] * num_copies, ignore_index=True)
    new_df_label = new_df_label.head(min_count-count)
    print(new_df_label.shape)
    new_dfs.append(new_df_label)

  # Concatenate the new DataFrames with the original DataFrame
  df_balanced = pd.concat([df_train] + new_dfs, ignore_index=True)
  return df_balanced

In [145]:
from sklearn.model_selection import train_test_split

#splitting the dataset and saving
if need_split_dataset==True:
  #dataset loading
  df = pd.read_csv(CollectedDatasetPath)[ ['Text','label'] ]
  print(f'df label counts\n',df['label'].value_counts())
  # check if there is any NaN value in the dataframe
  print(f'null values: {df.isna().sum()}')

  #null indices
  null_index = df.index[df.isna().any(axis=1)]
  print(f'null indices: {null_index}')

  #dropping null values
  df = df.dropna()

  df_train, df_val = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
  df_train.to_csv(SplittedTrainDataPath)
  df_val.to_csv(SplittedValDataPath)
else:
  df_train = pd.read_csv(SplittedTrainDataPath)[ ['text','label'] ]
  df_val = pd.read_csv(SplittedValDataPath)[ ['text','label'] ]

# count the number of each unique label in train and validation dataframes
train_label_counts = df_train['label'].value_counts()
val_label_counts = df_val['label'].value_counts()

#setting the first sample to be with label '0'
zero_index = df_train[df_train['label'] == 0].index[0]
first_index=0
# interchange the samples
df_train.iloc[[first_index, zero_index]] = df_train.iloc[[zero_index, first_index]]

print(df_train.shape)
print(df_val.shape)
print('Train label counts:\n', train_label_counts)
print('Validation label counts:\n', val_label_counts)

print("\n after making copies:")
#balance all classes making copies
df_train = balanceclasses(df_train)
print(df_train['label'].value_counts())

print(df_train)
print(df_val)

(2700, 2)
(1330, 2)
Train label counts:
 0    1389
1     922
2     389
Name: label, dtype: int64
Validation label counts:
 0    717
1    417
2    196
Name: label, dtype: int64

 after making copies:
(467, 2)
(1000, 2)
0    1389
1    1389
2    1389
Name: label, dtype: int64
                                                   text  label
0     এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদ...      0
1     যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...      1
2     আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...      2
3                                       দেশটা সুস্থ নাই      0
4     আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...      0
...                                                 ...    ...
4162  হাজার অভিজিৎ হত্যা হবে যদি কেউ নবীকে নিয়ে কটু...      2
4163  এগুলো মিথ্যা নিউজ।নিউমার্কেটের অসত দোকানদারদের...      2
4164         এরাই হলো দেশের কলঙ্ক এদেরকে মেরে ফেলা উচিত      2
4165  হরতালের নামে দেশে কোন অরাজকতা সৃষ্টি করতে চাইল...      2
4166  বাহ দারুন বিচার,,, আবরার (ম

In [8]:
class NewsDatasets(Dataset):
    def __init__(self, data, max_length=max_number_input_tokens):
        self.data = data

        self.config = {
            "max_length": max_length,
            "padding": "max_length",
            "return_tensors": "pt",
            "truncation": True,
            "add_special_tokens": True,
            "truncation_strategy":"longest_first"
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        value = self.data.iloc[idx]
        return value['text'] , value['label']

In [9]:
training_data = NewsDatasets(df_train)
train_dataloader = DataLoader(training_data, batch_size=batch_size_training, shuffle=True)

val_data = NewsDatasets(df_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size_training, shuffle=True)

test_data = NewsDatasets(df_val)
test_dataloader = DataLoader(test_data, batch_size=batch_size_training, shuffle=True)

In [10]:
class HateSpeechBert(nn.Module):

    def __init__(self, bert):
        super(HateSpeechBert, self).__init__()

        self.bert = bert

        # dropout layer
        self.dropout = nn.Dropout(first_dropout_rate)

        # relu activation function
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

        # dense layer 1
        self.fc1 = nn.Linear(hidden_output*2, hidden_output)

        #dense layer 2
        self.fc2 = nn.Linear(hidden_output, 128)

        # dense layer 2 (Output layer)
        self.fc3 = nn.Linear(128, 2)

        #softmax
        self.softmax = nn.Softmax(dim=1)

    # define the forward pass
    def forward(self, input_ids, token_type_ids, attention_mask):
        # pass the inputs to the model
        out = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        mean, _ = torch.max(out[0], 1)
        x= torch.cat((mean,out[1]), dim=1)

        x = self.dropout(x)

        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        # output layer
        x = self.fc3(x)
        x = self.softmax(x)

        return x

In [11]:
class BERTBengali(nn.Module):
    def __init__(self, bert):
        super(BERTBengali, self).__init__()
        #self.bert = BertForMaskedLM.from_pretrained("sagorsarker/bangla-bert-base")
        self.bert = bert
        self.bert_drop = nn.Dropout(0.2)
        self.out = nn.Linear(hidden_output, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        bo = self.bert_drop(output[1])

        output = self.out(bo)
        return output

In [12]:
class BERTBengaliPooler(nn.Module):
    def __init__(self, bert):
        super(BERTBengaliPooler, self).__init__()
        self.bert = bert
        #self.bert.pooler.dense = nn.Linear(bert.config.hidden_size, bert.config.hidden_size)
        self.bert_drop = nn.Dropout(first_dropout_rate)
        self.out = nn.Linear(bert.config.hidden_size, classes)
        #softmax
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled_output = outputs.pooler_output
        bo = self.bert_drop(pooled_output)

        output = self.out(bo)
        output = self.softmax(bo)
        return output

In [13]:
class CustomBERTBengali(nn.Module):
    def __init__(self, bert):
        super(CustomBERTBengali, self).__init__()
        self.bert = bert
        self.bert_drop = nn.Dropout(first_dropout_rate)
        self.tanh = nn.Tanh()
        self.out = nn.Linear(hidden_output * 3, classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        o1 = outputs.hidden_states[-1]
        o2 = outputs.pooler_output
        apool = torch.mean(o1, 1)
        mpool, _ = torch.max(o1, 1)
        pooled_output = o2
        cat = torch.cat((apool, mpool, pooled_output), 1)
        bo = self.bert_drop(cat)
        logits = self.out(bo)
        logits = self.softmax(logits)
        return logits

In [14]:
class BERTBengaliTwo(nn.Module):
    def __init__(self, bert):
        super(BERTBengaliTwo, self).__init__()
        self.bert = bert
        self.drop_out = nn.Dropout(first_dropout_rate)
        self.l0 =  nn.Linear(hidden_output * 2, classes)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        out = torch.cat((outputs.hidden_states[-1], outputs.hidden_states[-2]), dim=-1)
        out = self.drop_out(out)
        out = out[:,0,:]
        logits = self.l0(out)
        logits = self.softmax(logits)
        return logits

In [15]:
class BERTBengaliLastTwoPooler(nn.Module):
    def __init__(self, bert):
        super(BERTBengaliLastTwoPooler, self).__init__()
        self.bert = bert
        self.drop_out = nn.Dropout(first_dropout_rate)
        self.l0 =  nn.Linear(hidden_output * 3, classes)
        #torch.nn.init.normal_(self.l0.weight, std=0.02)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        mpool, _ = torch.max(outputs.hidden_states[-1], 1)
        out = torch.cat((mpool, outputs.hidden_states[-2][:,0,:],outputs.pooler_output), dim=-1)
        out = self.drop_out(out)
        #out = out[:,0,:]
        logits = self.l0(out)
        logits = self.softmax(logits)
        return logits

In [16]:
bert = BertModel.from_pretrained(bert_model_name, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#creating the structure to contain finetuned bert
struct_model = CustomBERTBengali(bert)
struct_model.to(device)

#loading the finetuned model which have leaned necessary info from other domain
struct_model.load_state_dict(torch.load(Finetuned_model_path))

# Access the bert model
finetuned_bert_base = struct_model.bert

<h1>Creating a model with transferred learned knowledge capable of being finetuned with collected data and training it
##Note: test data and val data are same here.<h1>

In [176]:
###Hyperparameter for the new model
#defining some hyperparameters
max_number_input_tokens=256
batch_size_training = 8
first_dropout_rate = 0.0
hidden_output = 768
bert_model_name = "sagorsarker/bangla-bert-base"
adam_opt_lr = 3e-5
scheduler_step = 1
scheduler_gamma = 0.98
epochs = 100
classes = 3
#need_split_dataset=False

In [177]:
class NewsDatasets(Dataset):
    def __init__(self, data, max_length=max_number_input_tokens):
        self.data = data

        self.config = {
            "max_length": max_length,
            "padding": "max_length",
            "return_tensors": "pt",
            "truncation": True,
            "add_special_tokens": True,
            "truncation_strategy":"longest_first"
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        value = self.data.iloc[idx]
        return value['text'] , value['label']

In [178]:
training_data = NewsDatasets(df_train)
train_dataloader = DataLoader(training_data, batch_size=batch_size_training, shuffle=False)

val_data = NewsDatasets(df_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size_training, shuffle=True)

test_data = NewsDatasets(df_val)
test_dataloader = DataLoader(test_data, batch_size=batch_size_training, shuffle=True)

In [179]:
# #model for finetuning collected data
# class BERTBengaliLastTwoPooler(nn.Module):
#     def __init__(self, bert):
#         super(BERTBengaliLastTwoPooler, self).__init__()
#         self.bert = bert
#         self.drop_out = nn.Dropout(first_dropout_rate)
#         self.l0 =  nn.Linear(hidden_output * 3, classes)
#         #torch.nn.init.normal_(self.l0.weight, std=0.02)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, input_ids, attention_mask, token_type_ids):
#         outputs = self.bert(
#             input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids
#         )
#         mpool, _ = torch.max(outputs.hidden_states[-1], 1)
#         out = torch.cat((mpool, outputs.hidden_states[-2][:,0,:],outputs.pooler_output), dim=-1)
#         out = self.drop_out(out)
#         #out = out[:,0,:]
#         logits = self.l0(out)
#         logits = self.softmax(logits)
#         return logits

In [180]:
# class CustomBERTBengali(nn.Module):
#     def __init__(self, bert):
#         super(CustomBERTBengali, self).__init__()
#         self.bert = bert
#         self.bert_drop = nn.Dropout(first_dropout_rate)
#         self.tanh = nn.Tanh()
#         self.out = nn.Linear(hidden_output * 3, classes)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, input_ids, attention_mask, token_type_ids):
#         outputs = self.bert(
#             input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids
#         )
#         o1 = outputs.hidden_states[-1]
#         o2 = outputs.pooler_output
#         apool = torch.mean(o1, 1)
#         mpool, _ = torch.max(o1, 1)
#         pooled_output = o2
#         cat = torch.cat((apool, mpool, pooled_output), 1)
#         bo = self.bert_drop(cat)
#         logits = self.out(bo)
#         #logits = self.softmax(logits)
#         return logits

In [181]:
#model for finetuning collected data
class BERTBengaliLastTwoPoolerFreeze(nn.Module):
    def __init__(self, bert):
        super(BERTBengaliLastTwoPoolerFreeze, self).__init__()
        self.bert = bert
        self.drop_out = nn.Dropout(first_dropout_rate)
        self.l2 = nn.Linear(hidden_output * 3, hidden_output * 2)
        self.activation = nn.Tanh()
        self.l1 = nn.Linear(hidden_output * 2, hidden_output * 2)
        self.activation = nn.Tanh()
        self.l0 = nn.Linear(hidden_output * 2, classes)
        #torch.nn.init.normal_(self.l0.weight, std=0.02)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        mpool, _ = torch.max(outputs.hidden_states[-1], 1)
        out = torch.cat((outputs.hidden_states[-2][:,0,:], mpool,outputs.pooler_output), dim=-1)
        out = self.drop_out(out)
        out = self.l2(out)
        out = self.activation(out)
        out = self.l1(out)
        out = self.activation(out)
        logits = self.l0(out)
        #prob = self.softmax(logits)
        return logits


In [182]:
#model for finetuning collected data
class BERTBengaliLastTwoPoolerFreezePrev(nn.Module):
    def __init__(self, bert):
        super(BERTBengaliLastTwoPoolerFreezePrev, self).__init__()
        self.bert = bert
        self.drop_out = nn.Dropout(first_dropout_rate)
        self.l1 = nn.Linear(hidden_output * 3, hidden_output * 2)
        self.activation = nn.Tanh()
        self.l0 = nn.Linear(hidden_output * 2, classes)
        #torch.nn.init.normal_(self.l0.weight, std=0.02)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        mpool, _ = torch.max(outputs.hidden_states[-1], 1)
        out = torch.cat((outputs.hidden_states[-2][:,0,:], mpool,outputs.pooler_output), dim=-1)
        out = self.drop_out(out)
        out = self.l1(out)
        out = self.activation(out)
        logits = self.l0(out)
        #prob = self.softmax(logits)
        return logits


In [183]:
model = BERTBengaliLastTwoPoolerFreeze(finetuned_bert_base)
# model2Forlastlayers = BERTBengaliLastTwoPoolerFreezePrev(finetuned_bert_base)

model.to(device)
# model2Forlastlayers.to(device)
# model2Forlastlayers.load_state_dict(torch.load(DirPath+bert_model_name+"_lasttwopoolerf_contest_val_from_acc1_frozen_acc1.pth"))

# model.l0 = model2Forlastlayers.l0
# model.l2 = model2Forlastlayers.l1
# model.bert = struct_model.bert

model.load_state_dict(torch.load(DirPath+bert_model_name+"_lasttwopoolerf_contest_val_from_finalhs_frozen_acc1.pth"))

for params in model.bert.parameters():
  params.requires_grad = False
for params in model.l2.parameters():
  params.requires_grad = True
for params in model.l1.parameters():
  params.requires_grad = True
for params in model.l0.parameters():
  params.requires_grad = True

for name, param in model.named_parameters():
  if param.requires_grad:
      print(f"name: {name} is trainable")
  else:
      print(f"name: {name} is non-trainable")

name: bert.embeddings.word_embeddings.weight is non-trainable
name: bert.embeddings.position_embeddings.weight is non-trainable
name: bert.embeddings.token_type_embeddings.weight is non-trainable
name: bert.embeddings.LayerNorm.weight is non-trainable
name: bert.embeddings.LayerNorm.bias is non-trainable
name: bert.encoder.layer.0.attention.self.query.weight is non-trainable
name: bert.encoder.layer.0.attention.self.query.bias is non-trainable
name: bert.encoder.layer.0.attention.self.key.weight is non-trainable
name: bert.encoder.layer.0.attention.self.key.bias is non-trainable
name: bert.encoder.layer.0.attention.self.value.weight is non-trainable
name: bert.encoder.layer.0.attention.self.value.bias is non-trainable
name: bert.encoder.layer.0.attention.output.dense.weight is non-trainable
name: bert.encoder.layer.0.attention.output.dense.bias is non-trainable
name: bert.encoder.layer.0.attention.output.LayerNorm.weight is non-trainable
name: bert.encoder.layer.0.attention.output.Laye

In [184]:
#testing if the input of model works before starting training
s = "আমি বাংলায় গান গাই। [SEP]"
t = tokenizer.encode_plus(s, return_tensors="pt").to(device)
print(t)
out = model(**t)
print(out)

{'input_ids': tensor([[  101,  2169,  2492,  9294,  2552, 13985,  1014,   102,   102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
tensor([[ 0.4510,  0.0456, -1.2028]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [185]:
from torch.optim.lr_scheduler import StepLR

optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=adam_opt_lr)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

In [186]:
def train(model, dataloader, optimizer, criterion, config):
    model.train()  # prep model for training
    train_loss = 0
    for batch in tqdm(dataloader):
        text, labels = batch

        model.zero_grad()

        inputs = tokenizer.batch_encode_plus(
            text, **config
        )
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        #labels = labels.to(device)
        labels = labels.to(device, dtype=torch.long)  # Convert labels to torch.long

        # move things to model
        logs = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(logs, labels)
        #print("successfully calculated criterion in train!")
        train_loss += loss.item() * input_ids.size(0)
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    return train_loss

In [187]:
def evaluate(model, dataloader, criterion, config):
    total = 0
    correct = 0
    valid_loss = 0.0
    label_0_TP = 0
    label_0_TN = 0
    label_0_FP = 0
    label_0_FN = 0

    label_1_TP = 0
    label_1_TN = 0
    label_1_FP = 0
    label_1_FN = 0

    label_2_TP = 0
    label_2_TN = 0
    label_2_FP = 0
    label_2_FN = 0

    model.eval()  # prep model for evaluation
    for batch in dataloader:
        text, labels = batch
        inputs = tokenizer.batch_encode_plus(
            text, **config
        )
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        labels = labels.to(device, dtype=torch.long)

        # move things to model
        output = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)

        loss_p = criterion(output, labels)
        # update running validation loss
        valid_loss += loss_p.item() * input_ids.size(0)
        # calculate accuracy
        proba = torch.exp(output)
        top_p, top_class = proba.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        # accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(f'predicted: {predicted} labels: {labels}')
        label_0_TP += ((predicted == 0) & (labels == 0)).sum().item()
        label_0_TN += ((predicted != 0) & (labels != 0)).sum().item()
        label_0_FP += ((predicted == 0) & (labels != 0)).sum().item()
        label_0_FN += ((predicted != 0) & (labels == 0)).sum().item()

        label_1_TP += ((predicted == 1) & (labels == 1)).sum().item()
        label_1_TN += ((predicted != 1) & (labels != 1)).sum().item()
        label_1_FP += ((predicted == 1) & (labels != 1)).sum().item()
        label_1_FN += ((predicted != 1) & (labels == 1)).sum().item()

        label_2_TP += ((predicted == 2) & (labels == 2)).sum().item()
        label_2_TN += ((predicted != 2) & (labels != 2)).sum().item()
        label_2_FP += ((predicted == 2) & (labels != 2)).sum().item()
        label_2_FN += ((predicted != 2) & (labels == 2)).sum().item()

    return total, correct, valid_loss, label_0_TP, label_0_TN, label_0_FP, label_0_FN, label_1_TP, label_1_TN, label_1_FP, label_1_FN, label_2_TP, label_2_TN, label_2_FP, label_2_FN


In [188]:

tokenizer_config = {
    "max_length": max_number_input_tokens,
    "padding": "max_length",
    "return_tensors": "pt",
    "truncation": True,
    "add_special_tokens": True,
     "truncation_strategy":"longest_first"
}

In [189]:
train_loss_data, valid_loss_data = [], []
valid_loss_min = np.Inf
since = time.time()
best_loss = np.inf
best_acc=0
sml = 1e-10
best_f1=0.4368

for epoch in range(epochs):


    if epoch==1:
      training_data = NewsDatasets(df_train)
      train_dataloader = DataLoader(training_data, batch_size=batch_size_training, shuffle=True)
    print("Epoch: {}/{}".format(epoch + 1, epochs))
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    total = 0
    correct = 0
    label_0_TP = 0
    label_0_TN = 0
    label_0_FP = 0
    label_0_FN = 0

    label_1_TP = 0
    label_1_TN = 0
    label_1_FP = 0
    label_1_FN = 0

    label_2_TP = 0
    label_2_TN = 0
    label_2_FP = 0
    label_2_FN = 0


    e_since = time.time()

    # Train Model
    train_loss += train(model, train_dataloader, optimizer, criterion, tokenizer_config)
    # Now Evaluate
    out = evaluate(model, val_dataloader, criterion, tokenizer_config)
    total += out[0]
    correct += out[1]
    valid_loss += out[2]
    label_0_TP += out[3]
    label_0_TN += out[4]
    label_0_FP += out[5]
    label_0_FN += out[6]

    label_1_TP += out[7]
    label_1_TN += out[8]
    label_1_FP += out[9]
    label_1_FN += out[10]

    label_2_TP += out[11]
    label_2_TN += out[12]
    label_2_FP += out[13]
    label_2_FN += out[14]

    # Calculate precision, recall, and F1-score for each class
    label_0_precision = label_0_TP / (label_0_TP + label_0_FP+sml)
    label_0_recall = label_0_TP / (label_0_TP + label_0_FN+sml)
    label_0_f1_score = 2 * (label_0_precision * label_0_recall) / (label_0_precision + label_0_recall+sml)

    label_1_precision = label_1_TP / (label_1_TP + label_1_FP+sml)
    label_1_recall = label_1_TP / (label_1_TP + label_1_FN+sml)
    label_1_f1_score = 2 * (label_1_precision * label_1_recall) / (label_1_precision + label_1_recall+sml)

    label_2_precision = label_2_TP / (label_2_TP + label_2_FP+sml)
    label_2_recall = label_2_TP / (label_2_TP + label_2_FN+sml)
    label_2_f1_score = 2 * (label_2_precision * label_2_recall) / (label_2_precision + label_2_recall+sml)

    # Calculate combined F1-score
    combined_f1_score = (label_0_f1_score + label_1_f1_score + label_2_f1_score) / 3

    # Calculate micro TP, TN, FP, FN values
    micro_TP = label_0_TP + label_1_TP + label_2_TP
    micro_TN = label_0_TN + label_1_TN + label_2_TN
    micro_FP = label_0_FP + label_1_FP + label_2_FP
    micro_FN = label_0_FN + label_1_FN + label_2_FN

    # Calculate micro precision, recall, and F1 score
    micro_precision = micro_TP / (micro_TP + micro_FP)
    micro_recall = micro_TP / (micro_TP + micro_FN)
    micro_f1 = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

    scheduler.step()

    # print training/validation statistics
    # calculate average loss over an epoch
    train_loss = train_loss / len(train_dataloader.dataset)
    valid_loss = valid_loss / len(val_dataloader.dataset)

    val_acc=correct / total * 100

    # calculate train loss and running loss
    train_loss_data.append(train_loss * 100)
    valid_loss_data.append(valid_loss * 100)

    if combined_f1_score > best_f1:
        best_f1 = combined_f1_score
        torch.save(model.state_dict(), DirPath+bert_model_name+"_lasttwopoolerf_contest_val_from_finalhs_frozen_acc1_step2.pth")
        print(f'saved on epoch: {epoch+1}')

    print("\tTrain loss:{:.6f}..".format(train_loss),
          "\tValid Loss:{:.6f}..".format(valid_loss),
          "\tVal Accuracy: {:.4f}".format(correct / total * 100))
    print("\tLabel 0 Precision: {:.4f}\tLabel 0 Recall: {:.4f}\tLabel 0 F1-score: {:.4f}\n"
      "\tLabel 1 Precision: {:.4f}\tLabel 1 Recall: {:.4f}\tLabel 1 F1-score: {:.4f}\n"
      "\tLabel 2 Precision: {:.4f}\tLabel 2 Recall: {:.4f}\tLabel 2 F1-score: {:.4f}\n"
      "\tCombined F1-score: {:.4f}".format(label_0_precision, label_0_recall, label_0_f1_score,
                                            label_1_precision, label_1_recall, label_1_f1_score,
                                            label_2_precision, label_2_recall, label_2_f1_score,
                                            combined_f1_score))
    print(f'micro precision: {micro_precision}, Micro recall: {micro_recall}, micro f1: {micro_f1}')

time_elapsed = time.time() - since
print('Training completed in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

Epoch: 1/100


  0%|          | 0/521 [00:00<?, ?it/s]

	Train loss:0.936735.. 	Valid Loss:5.841118.. 	Val Accuracy: 14.7368
	Label 0 Precision: 0.0000	Label 0 Recall: 0.0000	Label 0 F1-score: 0.0000
	Label 1 Precision: 0.0000	Label 1 Recall: 0.0000	Label 1 F1-score: 0.0000
	Label 2 Precision: 0.1474	Label 2 Recall: 1.0000	Label 2 F1-score: 0.2569
	Combined F1-score: 0.0856
micro precision: 0.14736842105263157, Micro recall: 0.14736842105263157, micro f1: 0.14736842105263157
Epoch: 2/100


  0%|          | 0/521 [00:00<?, ?it/s]

saved on epoch: 2
	Train loss:1.261723.. 	Valid Loss:1.046717.. 	Val Accuracy: 46.6917
	Label 0 Precision: 0.6139	Label 0 Recall: 0.4923	Label 0 F1-score: 0.5464
	Label 1 Precision: 0.4381	Label 1 Recall: 0.4077	Label 1 F1-score: 0.4224
	Label 2 Precision: 0.2670	Label 2 Recall: 0.5000	Label 2 F1-score: 0.3481
	Combined F1-score: 0.4390
micro precision: 0.46691729323308273, Micro recall: 0.46691729323308273, micro f1: 0.46691729323308273
Epoch: 3/100


...

	Train loss:1.026890.. 	Valid Loss:1.050372.. 	Val Accuracy: 47.6692
	Label 0 Precision: 0.5598	Label 0 Recall: 0.6722	Label 0 F1-score: 0.6109
	Label 1 Precision: 0.4302	Label 1 Recall: 0.1775	Label 1 F1-score: 0.2513
	Label 2 Precision: 0.2626	Label 2 Recall: 0.3980	Label 2 F1-score: 0.3164
	Combined F1-score: 0.3929
micro precision: 0.4766917293233083, Micro recall: 0.4766917293233083, micro f1: 0.4766917293233083
Epoch: 19/100


  0%|          | 0/521 [00:00<?, ?it/s]

	Train loss:1.028898.. 	Valid Loss:1.071393.. 	Val Accuracy: 43.4586
	Label 0 Precision: 0.5624	Label 0 Recall: 0.6220	Label 0 F1-score: 0.5907
	Label 1 Precision: 0.3718	Label 1 Recall: 0.0695	Label 1 F1-score: 0.1172
	Label 2 Precision: 0.2244	Label 2 Recall: 0.5255	Label 2 F1-score: 0.3145
	Combined F1-score: 0.3408
micro precision: 0.4345864661654135, Micro recall: 0.4345864661654135, micro f1: 0.4345864661654135
Epoch: 20/100


  0%|          | 0/521 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [175]:
torch.cuda.empty_cache()

In [ ]:
from matplotlib import pyplot as plt

plt.plot(train_loss_data, label="Training loss")
plt.plot(valid_loss_data, label="validation loss")
plt.legend(frameon=False)

In [ ]:
model.load_state_dict(torch.load(DirPath+bert_model_name+"_lasttwopooler_contest_val.pth", map_location = device))

<All keys matched successfully>

In [ ]:
all_preds = []
all_labels = []

for batch in test_dataloader:
    text, labels = batch
    inputs = tokenizer.batch_encode_plus(
        text, **tokenizer_config
    )
    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    labels = labels.to(device)

    # move things to model
    output = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)
    preds = output.detach().cpu().numpy()
    preds = np.argmax(preds, axis = 1)
    all_preds.extend(preds)
    all_labels.extend(labels.cpu().numpy())

In [ ]:
from sklearn.metrics import classification_report

# preds = np.argmax(preds, axis = 1)
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89      2613
           1       0.90      0.85      0.88      2416

    accuracy                           0.89      5029
   macro avg       0.89      0.88      0.88      5029
weighted avg       0.89      0.89      0.89      5029



<h1>Training the model with All Collected dataset with the selected model and hyperparameters(Code not yet updated)<h1>

In [ ]:
!pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.4 MB/s eta 0:00:00


In [ ]:
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

In [ ]:
#df loading
df_train = pd.read_csv('train.csv')[['sentence','hate speech']]
df_val = pd.read_csv('val.csv')[['sentence','hate speech']]
df_test = pd.read_csv('test.csv')[['sentence','hate speech']]

#concatenating all the data
df_train = pd.concat([df_train, df_val, df_test], ignore_index=True)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)
print(df_train)
print(df_train.describe())

(50281, 2)
(5028, 2)
(5029, 2)
                                                sentence  hate speech
0                             .... ঐ ইন্দুর তোই মরছ নাই?            1
1      #গেবনের শেষে আইসা আপনার মুখোশ টা খুলছে এতেই আম...            1
2            ✈✈✈✈��� মুরগি চোরের পাছায় ডুকবি আর মারবি।।।            1
3      ১৮ কোটির চোদা খাওয়া শেষে এখন ১৫০কোটির চোদা খাব...            1
4                          ২য় মীর জাফরের মুখে মুতে দে...            1
...                                                  ...          ...
50276  হেট লাইন কি দিছোফালতু হেট লাইন দেয়ার জন্যআইন অ...            0
50277  হ্যা অবশ্যই পারবে শাকিব খান যেমন সুন্দর স্মার্...            0
50278                  হ্যাঁ ক্ষমা করে দিয়েছি রানু দিকে            0
50279  হ্যাঁ পমিতের যাব যেন শাকিব খানের উপজেলা এক হয়...            0
50280  হ্যালো আসসালামু আলাইকুম আল্লাহ তুমি মুসলমানদের...            0

[50281 rows x 2 columns]
        hate speech
count  50281.000000
mean       0.480420
std        0.499621
min        0.000000
25%

In [ ]:
#defining previous hyperparameters got from testing
max_number_input_tokens=256
batch_size_training = 16
first_dropout_rate = 0.3
hidden_output = 768
bert_model_name = "sagorsarker/bangla-bert-base"
adam_opt_lr = 3e-5
scheduler_step = 1
scheduler_gamma = 0.8
epochs = 6
classes = 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DirPath = ('/content/drive/My Drive/Test/')

Mounted at /content/drive


In [ ]:
class NewsDatasets(Dataset):
    def __init__(self, data, max_length=max_number_input_tokens):
        self.data = data

        self.config = {
            "max_length": max_length,
            "padding": "max_length",
            "return_tensors": "pt",
            "truncation": True,
            "add_special_tokens": True,
            "truncation_strategy":"longest_first"
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        value = self.data.iloc[idx]
        return value['sentence'] , value['hate speech']

In [ ]:
training_data = NewsDatasets(df_train)
train_dataloader = DataLoader(training_data, batch_size=batch_size_training, shuffle=True)

val_data = NewsDatasets(df_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size_training, shuffle=True)

test_data = NewsDatasets(df_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size_training, shuffle=True)

In [ ]:
class CustomBERTBengali(nn.Module):
    def __init__(self, bert):
        super(CustomBERTBengali, self).__init__()
        self.bert = bert
        self.bert_drop = nn.Dropout(first_dropout_rate)
        self.tanh = nn.Tanh()
        self.out = nn.Linear(hidden_output * 3, classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        o1 = outputs.hidden_states[-1]
        o2 = outputs.pooler_output
        apool = torch.mean(o1, 1)
        mpool, _ = torch.max(o1, 1)
        pooled_output = o2
        cat = torch.cat((apool, mpool, pooled_output), 1)
        bo = self.bert_drop(cat)
        logits = self.out(bo)
        logits = self.softmax(logits)
        return logits

In [ ]:
bert = BertModel.from_pretrained(bert_model_name, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomBERTBengali(bert)
model.to(device)

CustomBERTBengali(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(102025, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
from torch.optim.lr_scheduler import StepLR

optimizer = AdamW(model.parameters(), lr=adam_opt_lr)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

In [ ]:
def train(model, dataloader, optimizer, criterion, config):
    model.train()  # prep model for training
    train_loss = 0
    for batch in tqdm(dataloader):
        text, labels = batch

        model.zero_grad()

        inputs = tokenizer.batch_encode_plus(
            text, **config
        )
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        #labels = labels.to(device)
        labels = labels.to(device, dtype=torch.long)  # Convert labels to torch.long

        # move things to model
        logs = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(logs, labels)
        train_loss += loss.item() * input_ids.size(0)
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    return train_loss

In [ ]:
def evaluate(model, dataloader, criterion, config):
    total = 0
    correct = 0
    valid_loss = 0.0

    model.eval()  # prep model for evaluation
    for batch in dataloader:
        text, labels = batch
        inputs = tokenizer.batch_encode_plus(
            text, **config
        )
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        labels = labels.to(device)

        # move things to model
        output = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)

        loss_p = criterion(output, labels)
        # update running validation loss
        valid_loss += loss_p.item() * input_ids.size(0)
        # calculate accuracy
        proba = torch.exp(output)
        top_p, top_class = proba.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        # accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return total, correct, valid_loss

In [ ]:
tokenizer_config = {
    "max_length": max_number_input_tokens,
    "padding": "max_length",
    "return_tensors": "pt",
    "truncation": True,
    "add_special_tokens": True,
     "truncation_strategy":"longest_first"
}

In [ ]:
train_loss_data, valid_loss_data = [], []
valid_loss_min = np.Inf
since = time.time()
best_loss = np.inf

for epoch in range(epochs):
    print("Epoch: {}/{}".format(epoch + 1, epochs))
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    total = 0
    correct = 0
    e_since = time.time()

    # Train Model
    train_loss += train(model, train_dataloader, optimizer, criterion, tokenizer_config)
    # Now Evaluate
    out = evaluate(model, val_dataloader, criterion, tokenizer_config)
    total += out[0]
    correct += out[1]
    valid_loss += out[2]

    scheduler.step()

    # print training/validation statistics
    # calculate average loss over an epoch
    train_loss = train_loss / len(train_dataloader.dataset)
    valid_loss = valid_loss / len(val_dataloader.dataset)

    # calculate train loss and running loss
    train_loss_data.append(train_loss * 100)
    valid_loss_data.append(valid_loss * 100)

    if True:
        best_loss = valid_loss
        torch.save(model.state_dict(), DirPath+bert_model_name+"_CustomBertBengaliFullDataset6epoch885044valacc.pth")
        print(f'epoch: {epoch+1}')

    print("\tTrain loss:{:.6f}..".format(train_loss),
          "\tValid Loss:{:.6f}..".format(valid_loss),
          "\tVal Accuracy: {:.4f}".format(correct / total * 100))

time_elapsed = time.time() - since
print('Training completed in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

Epoch: 1/6


  0%|          | 0/3143 [00:00<?, ?it/s]

epoch: 1
	Train loss:0.449559.. 	Valid Loss:0.420331.. 	Val Accuracy: 89.2601
Epoch: 2/6


  0%|          | 0/3143 [00:00<?, ?it/s]

epoch: 2
	Train loss:0.432832.. 	Valid Loss:0.430235.. 	Val Accuracy: 88.2458
Epoch: 3/6


...

epoch: 4
	Train loss:0.439238.. 	Valid Loss:0.435497.. 	Val Accuracy: 87.7685
Epoch: 5/6


  0%|          | 0/3143 [00:00<?, ?it/s]

epoch: 5
	Train loss:0.448464.. 	Valid Loss:0.429014.. 	Val Accuracy: 88.4248
Epoch: 6/6


  0%|          | 0/3143 [00:00<?, ?it/s]

epoch: 6
	Train loss:0.444639.. 	Valid Loss:0.436770.. 	Val Accuracy: 87.6492
Training completed in 227m 4s
